In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import warnings
from optuna.exceptions import ExperimentalWarning
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=ExperimentalWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

In [3]:
import numpy as np
import pandas as pd
import time
from tqdm import tqdm
import matplotlib.pyplot as plt
import psutil


# from pyspark.sql import SparkSession
from replay.utils.session_handler import get_spark_session, State

from replay.models import UCB, Wilson, RandomRec, LinUCB, ThompsonSampling, LinTS
from replay.experimental.scenarios.movielens_wrapper.replay_offline import OBPOfflinePolicyLearner
from replay.experimental.scenarios.movielens_wrapper.dataset import MovielensBanditDataset
from replay.experimental.scenarios.movielens_wrapper.utils import get_est_rewards_by_reg, bandit_subset
from replay.utils.spark_utils import convert2spark

'nlp' extra dependecy package 'gensim' isn't installed. Look at README.md in repo 'LightAutoML' for installation instructions.
'nlp' extra dependecy package 'nltk' isn't installed. Look at README.md in repo 'LightAutoML' for installation instructions.
'nlp' extra dependecy package 'transformers' isn't installed. Look at README.md in repo 'LightAutoML' for installation instructions.
'nlp' extra dependecy package 'gensim' isn't installed. Look at README.md in repo 'LightAutoML' for installation instructions.
'nlp' extra dependecy package 'nltk' isn't installed. Look at README.md in repo 'LightAutoML' for installation instructions.
'nlp' extra dependecy package 'transformers' isn't installed. Look at README.md in repo 'LightAutoML' for installation instructions.


In [4]:
spark = State(get_spark_session()).session
spark.sparkContext.setLogLevel('ERROR')

24/10/04 11:40:55 WARN Utils: Your hostname, hdilab-hdilabALIEN05 resolves to a loopback address: 127.0.1.1; using 172.21.136.110 instead (on interface enp3s0)
24/10/04 11:40:55 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/10/04 11:40:55 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/10/04 11:40:56 WARN SparkConf: Note that spark.local.dir will be overridden by the value set by the cluster manager (via SPARK_LOCAL_DIRS in mesos/standalone/kubernetes and LOCAL_DIRS in YARN).


Lets define OpenBanditDataset class with random policy. For the purpose of demonstration we won't use the whole dataset but only subset of size 10000.

In [5]:
from rs_datasets import MovieLens

data = MovieLens("1m")

Оставим только 300 самых популярных айтемов

In [6]:
logs = data.ratings
logs['cnt'] = 1
logs = logs[['item_id', 'cnt']].groupby(by=["item_id"]).sum()
logs = logs.sort_values(by=['cnt'], ascending=False).reset_index()
popular_items = logs.iloc[:1500]['item_id'].tolist()
# popular_items = logs['item_id'].tolist()

In [7]:
data.ratings = data.ratings[data.ratings['item_id'].isin(popular_items)]
print('размер датасета логов после выброса непопулярных айтемов:', data.ratings.shape)

размер датасета логов после выброса непопулярных айтемов: (869950, 5)


Оставим информацию только о фичах юзеров и айтемов которые остались в логах

In [8]:
data.items = data.items[data.items['item_id'].isin(popular_items)]
users = set(data.ratings['user_id'].tolist())
data.users = data.users[data.users['user_id'].isin(users)]
data.users.shape, data.items.shape

((6040, 5), (1500, 3))

In [9]:
dataset = MovielensBanditDataset(data)

04-Oct-24 11:41:01, replay, INFO: Columns with ids of users or items are present in mapping. The dataframe will be treated as an interactions log.
INFO:replay:Columns with ids of users or items are present in mapping. The dataframe will be treated as an interactions log.


04-Oct-24 11:41:07, replay, INFO: Column with ids of users or items is absent in mapping. The dataframe will be treated as a users'/items' features dataframe.
INFO:replay:Column with ids of users or items is absent in mapping. The dataframe will be treated as a users'/items' features dataframe.
04-Oct-24 11:41:08, replay, INFO: Column with ids of users or items is absent in mapping. The dataframe will be treated as a users'/items' features dataframe.
INFO:replay:Column with ids of users or items is absent in mapping. The dataframe will be treated as a users'/items' features dataframe.


fit started
predict started


100%|██████████| 869950/869950 [01:52<00:00, 7703.02it/s]


# Metrics evaluation

In [10]:
import warnings
warnings.filterwarnings("ignore")

In [11]:
bandit_feedback_train, bandit_feedback_test = dataset.obtain_batch_bandit_feedback(test_size=0.3, is_timeseries_split=True)

In [12]:
print('momory usage:', psutil.virtual_memory().percent)
    
print('UCB')
model_1 = UCB(exploration_coef = 2, sample = True, seed = 123)
learner_1 = OBPOfflinePolicyLearner(n_actions=dataset.n_actions,
                                    replay_model=model_1)
learner_1.fit(bandit_feedback_train)

print(learner_1.predict_and_evaluate_new(bandit_feedback_test, 1))
print(learner_1.predict_and_evaluate_new(bandit_feedback_test, 3))
print(learner_1.predict_and_evaluate_new(bandit_feedback_test, 10))

momory usage: 20.0
UCB


100%|██████████| 229/229 [00:59<00:00,  3.88it/s]


{'hr@1': 0.003137472718287669, 'mrr@1': 0.003137472718287669, 'ndcg@1': 0.003137472718287669, 'cov@1': 0.005333333333333333}


100%|██████████| 229/229 [00:54<00:00,  4.19it/s]


{'hr@3': 0.005252306828513527, 'mrr@3': 0.004093457213400368, 'ndcg@3': 0.004392101241787269, 'cov@3': 0.01}


100%|██████████| 229/229 [00:53<00:00,  4.29it/s]


{'hr@10': 0.00806021547926409, 'mrr@10': 0.00452776810467312, 'ndcg@10': 0.005355937743317153, 'cov@10': 0.027333333333333334}


In [13]:
print('momory usage:', psutil.virtual_memory().percent)
    
print('LinUCB')
model_2 = LinUCB(eps = -10.0, alpha = 1.0, regr_type = 'disjoint')
learner_2 = OBPOfflinePolicyLearner(n_actions=dataset.n_actions,
                                    replay_model=model_2)
learner_2.fit(bandit_feedback_train)

print(learner_2.predict_and_evaluate_new(bandit_feedback_test, 1))
print(learner_2.predict_and_evaluate_new(bandit_feedback_test, 3))
print(learner_2.predict_and_evaluate_new(bandit_feedback_test, 10))

momory usage: 27.1
LinUCB


[autoreload of replay.models.ucb failed: Traceback (most recent call last):
  File "/home/hdilab/sudakovcom/.conda/lib/python3.9/site-packages/IPython/extensions/autoreload.py", line 273, in check
    superreload(m, reload, self.old_objects)
  File "/home/hdilab/sudakovcom/.conda/lib/python3.9/site-packages/IPython/extensions/autoreload.py", line 496, in superreload
    update_generic(old_obj, new_obj)
  File "/home/hdilab/sudakovcom/.conda/lib/python3.9/site-packages/IPython/extensions/autoreload.py", line 393, in update_generic
    update(a, b)
  File "/home/hdilab/sudakovcom/.conda/lib/python3.9/site-packages/IPython/extensions/autoreload.py", line 345, in update_class
    if update_generic(old_obj, new_obj):
  File "/home/hdilab/sudakovcom/.conda/lib/python3.9/site-packages/IPython/extensions/autoreload.py", line 393, in update_generic
    update(a, b)
  File "/home/hdilab/sudakovcom/.conda/lib/python3.9/site-packages/IPython/extensions/autoreload.py", line 305, in update_function


{'hr@1': 0.009587229125275192, 'mrr@1': 0.009587229125275192, 'ndcg@1': 0.009587229125275192, 'cov@1': 0.048}


100%|██████████| 229/229 [00:25<00:00,  8.94it/s]


{'hr@3': 0.024031845133832556, 'mrr@3': 0.015771024940098088, 'ndcg@3': 0.017884934273710216, 'cov@3': 0.08733333333333333}


100%|██████████| 229/229 [00:25<00:00,  8.97it/s]


{'hr@10': 0.06216490458087979, 'mrr@10': 0.02186516577173348, 'ndcg@10': 0.031147566431041614, 'cov@10': 0.17}


In [14]:
print('momory usage:', psutil.virtual_memory().percent)

print('Random')
model_3 = RandomRec(seed=42)
learner_3 = OBPOfflinePolicyLearner(n_actions=dataset.n_actions,
                                    replay_model=model_3)
learner_3.fit(bandit_feedback_train)

print(learner_3.predict_and_evaluate_new(bandit_feedback_test, 1))
print(learner_3.predict_and_evaluate_new(bandit_feedback_test, 3))
print(learner_3.predict_and_evaluate_new(bandit_feedback_test, 10))

momory usage: 34.1
Random


100%|██████████| 229/229 [00:41<00:00,  5.49it/s]


{'hr@1': 0.0011735538385816873, 'mrr@1': 0.0011735538385816873, 'ndcg@1': 0.0011735538385816873, 'cov@1': 0.7926666666666666}


100%|██████████| 229/229 [00:41<00:00,  5.54it/s]


{'hr@3': 0.0025322369116594088, 'mrr@3': 0.001754318802703772, 'ndcg@3': 0.00195334777705364, 'cov@3': 0.994}


100%|██████████| 229/229 [00:41<00:00,  5.54it/s]


{'hr@10': 0.007225143171832117, 'mrr@10': 0.0024818648864348277, 'ndcg@10': 0.0035642191737491817, 'cov@10': 1.0}


In [15]:
print('momory usage:', psutil.virtual_memory().percent)
    
print('LinTS')
model_4 = LinTS(nu = 0.114225, alpha = 1016.561481, regr_type = 'disjoint')
learner_4 = OBPOfflinePolicyLearner(n_actions=dataset.n_actions,
                                    replay_model=model_4)
learner_4.fit(bandit_feedback_train)

print(learner_4.predict_and_evaluate_new(bandit_feedback_test, 1))
print(learner_4.predict_and_evaluate_new(bandit_feedback_test, 3))
print(learner_4.predict_and_evaluate_new(bandit_feedback_test, 10))

momory usage: 39.8
LinTS


100%|██████████| 229/229 [01:38<00:00,  2.32it/s]


{'hr@1': 0.006187871398840221, 'mrr@1': 0.006187871398840221, 'ndcg@1': 0.006187871398840221, 'cov@1': 0.064}


100%|██████████| 229/229 [01:38<00:00,  2.33it/s]


{'hr@3': 0.013962941006144041, 'mrr@3': 0.00865714815113251, 'ndcg@3': 0.010014675104411638, 'cov@3': 0.176}


100%|██████████| 229/229 [01:38<00:00,  2.33it/s]


{'hr@10': 0.03548643938533158, 'mrr@10': 0.010791719269673072, 'ndcg@10': 0.016453397191911752, 'cov@10': 0.37}
